# RAG評価用データセットの生成
* Ragasでは、評価用のデータセットを生成できる
* 以下、作成方法
* 書籍のバージョンではデータセットに設定するキー名が旧版のためエラーがでるので注意

In [ ]:
# !pip install numpy==1.26.4

In [ ]:
# !pip install langchain-core==0.2.30 langchain-openai==0.1.21 \
#     langchain-community==0.2.12 GitPython==3.1.43 \
#     langchain-chroma==0.1.2 chromadb==0.5.3 \
#     ragas==0.1.14 nest-asyncio==1.6.0 pydantic==2.10.6

In [ ]:
# # 環境変数(.env)
# OPENAI_API_KEY=
# LANGCHAIN_API_KEY=
# LANGCHAIN_PROJECT=

In [ ]:
import os
import nest_asyncio
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

In [ ]:
### ベクトル検索対象データ用意
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="langchain==0.2.13",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

## * Ragasで合成データ作成

In [ ]:
for document in documents:
    document.metadata["filename"] = document.metadata["source"]


nest_asyncio.apply()

generator = TestsetGenerator.from_langchain(
    generator_llm=ChatOpenAI(model="gpt-4o-mini"),
    critic_llm=ChatOpenAI(model="gpt-4o-mini"),
    embeddings=OpenAIEmbeddings(),
)

testset = generator.generate_with_langchain_docs(
    documents,
    test_size=4,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

testset.to_pandas()

### * LangSmithのDataSet作成
* LangSmithには、評価用の「Dataset」を管理する機能がある。

#### 以下は、LangSmithでデータセットを管理する「Dataset」オブジェクトを作成するコード

In [ ]:
from langsmith import Client

dataset_name = "test-dataset-2"

client = Client()

if client.has_dataset(dataset_name=dataset_name):
    client.delete_dataset(dataset_name=dataset_name)

dataset = client.create_dataset(
    dataset_name=dataset_name,
)

上記実行後、LangSmithの画面で、『Datasets & Experiments』を選ぶと"test-dataset"というデータセットができる。ここに、Ragasで生成した合成テストデータを保存する

### * 合成テストデータの保存
* まずは、生成したデータセットをLangSmithのDatasetに保存する形式に変換

In [ ]:
inputs = []
outputs = []
metadatas = []

for testset_record in testset.test_data:
    inputs.append(
        {
            # "question": testset_record.question, # 古いバージョン
            "user_input": testset_record.question,
        }
    )
    outputs.append(
        {
            # "contexts": testset_record.contexts,
            # "ground_truth": testset_record.ground_truth,
            "retrieved_contexts": testset_record.contexts,      # ← contexts → retrieved_contexts
            "response": testset_record.ground_truth,            # ← ground_truth → response
            "reference": testset_record.ground_truth,           # ← 必要に応じて reference も登録
        }
    )
    metadatas.append(
        {
            "source": testset_record.metadata[0]["source"],
            "evolution_type": testset_record.evolution_type,
        }
    )

### * LangSmithのクライアントを使用して、DatasetのIDを指定しデータを保存
* LangSmithでは、Datasetに保存するデータの１件１件を「Example」と呼ぶ。


In [ ]:
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    metadatas=metadatas,
    dataset_id=dataset.id,
)

* これにより、web画面で確認したDataset（"test-dataset")を選ぶと、登録されているのがわかる